# **Autoencoder**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

from src.autoencoder.autoencoder import ConvAutoencoder

In [3]:
encoder_channels = [1, 16, 32, 64]
decoder_channels = [64, 32, 16, 1]
autoencoder = ConvAutoencoder(encoder_channels, decoder_channels)

In [ ]:
# read pussy.jpg
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('../data/pussy.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (128, 128))
plt.imshow(img, cmap='gray')
plt.show()

: 